# ResNet for MNIST in PyTorch

In [1]:
from torchvision.models.resnet import ResNet, BasicBlock
from torchvision.datasets import MNIST
from tqdm.autonotebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import inspect
import time

import torch

from torch import nn, optim
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from torch.utils.data import DataLoader
from copy import copy, deepcopy
import numpy as np


In [2]:
# model = models.resnet152(pretrained=True)
# for param in model.parameters():
#     param.requires_grad = False

In [3]:

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
def save_last_model(input_model):
  model_save_name = 'resnet_sgd_last.pkl'
  path = F"/content/gdrive/My Drive/{model_save_name}" 
  torch.save(input_model, path)

In [5]:
class MnistResNet(ResNet):
    def __init__(self):
        super(MnistResNet, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=10)
        self.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
    def forward(self, x):
        return torch.softmax(super(MnistResNet, self).forward(x), dim=-1)


In [6]:
# def get_data_loaders(train_batch_size, val_batch_size):
#     mnist = MNIST(download=True, train=True, root=".").train_data.float()
    
#     # add gaussian noise maybe
#     data_transform = Compose([ Resize((224, 224)),ToTensor(), Normalize((mnist.mean()/255,), (mnist.std()/255,))])

#     train_loader = DataLoader(MNIST(download=True, root=".", transform=data_transform, train=True),
#                               batch_size=train_batch_size, shuffle=True)

#     val_loader = DataLoader(MNIST(download=False, root=".", transform=data_transform, train=False),
#                             batch_size=val_batch_size, shuffle=False)
#     return train_loader, val_loader
def getData(name='cifar10', train_bs=128, test_bs=1000):    
    if name == 'svhn':
        train_loader = torch.utils.data.DataLoader(
    datasets.SVHN('../data', split='extra', download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=train_bs, shuffle=True)
        test_loader = torch.utils.data.DataLoader(
    datasets.SVHN('../data', split='test', download=True,transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=test_bs, shuffle=False)
    if name == 'mnist':
        train_loader = torch.utils.data.DataLoader(
            datasets.MNIST('../data', train=True, download=True,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ])),
            batch_size=train_bs, shuffle=True)
        test_loader = torch.utils.data.DataLoader(
            datasets.MNIST('../data', train=False, transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ])),
            batch_size=test_bs, shuffle=False)
    if name == 'emnist':
        train_loader = torch.utils.data.DataLoader(
            datasets.EMNIST('../data', train=True, download=True, split='balanced',
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1751,), (0.3267,))
                           ])),
            batch_size=train_bs, shuffle=True)
        test_loader = torch.utils.data.DataLoader(
            datasets.EMNIST('../data', train=False, split='balanced', transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1751,), (0.3267,))
                           ])),
            batch_size=test_bs, shuffle=False)
    if name == 'cifar10':
        transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
        transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
        trainset = datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)
        train_loader = torch.utils.data.DataLoader(trainset, batch_size=train_bs, shuffle=True)
        testset = datasets.CIFAR10(root='../data', train=False, download=False, transform=transform_test)
        test_loader = torch.utils.data.DataLoader(testset, batch_size=test_bs, shuffle=False)
    if name == 'cifar100':
        transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
        transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
        trainset = datasets.CIFAR100(root='../data', train=True, download=True, transform=transform_train)
        train_loader = torch.utils.data.DataLoader(trainset, batch_size=train_bs, shuffle=True)
        testset = datasets.CIFAR100(root='../data', train=False, download=False, transform=transform_test)
        test_loader = torch.utils.data.DataLoader(testset, batch_size=test_bs, shuffle=False)
    if name == 'tinyimagenet':      
        normalize = transforms.Normalize(mean=[0.44785526394844055, 0.41693055629730225, 0.36942949891090393],
                                     std=[0.2928885519504547, 0.28230994939804077, 0.2889912724494934])
        train_dataset = datasets.ImageFolder(
        '../data/tiny-imagenet-200/train',
        transforms.Compose([
            transforms.RandomCrop(64, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_bs, shuffle=True, num_workers=4, pin_memory=False)
        test_dataset = datasets.ImageFolder(
        '../data/tiny-imagenet-200/val',
        transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ]))
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_bs, shuffle=False)
    return train_loader, test_loader

In [7]:
def calculate_metric(metric_fn, true_y, pred_y):
    if "average" in inspect.getfullargspec(metric_fn).args:
        return metric_fn(true_y, pred_y, average="macro")
    else:
        return metric_fn(true_y, pred_y)
    
def print_scores(p, r, f1, a, batch_size):
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")

In [8]:
def exp_lr_scheduler(epoch, optimizer, strategy='normal', decay_eff=0.1, decayEpoch=[]):
    """Decay learning rate by a factor of lr_decay every lr_decay_epoch epochs"""
    if strategy == 'normal':
        if epoch in decayEpoch:
            for param_group in optimizer.param_groups:
                param_group['lr'] *= decay_eff
            print('New learning rate is: ', param_group['lr'])
    else:
        print('wrong strategy')
        raise ValueError('A very specific bad thing happened.')
    return optimizer

In [11]:
start_ts = time.time()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epochs = 90

model = MnistResNet().to(device)
train_loader, val_loader = getData(name='mnist', train_bs=128, test_bs=1000)

losses = []
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

batches = len(train_loader)
val_batches = len(val_loader)

# keep best model
accuracies=[]
best_accuracy = 0
best_model = deepcopy(model)

# training loop + eval loop
for epoch in range(epochs):
    total_loss = 0
    progress = tqdm(enumerate(train_loader), desc="Loss: ", total=batches)
    model.train()
    # lr decay
    optimizer = exp_lr_scheduler(epoch, optimizer, decay_eff=0.1, decayEpoch=[30,60,80])


    for i, data in progress:
        X, y = data[0].to(device), data[1].to(device)
        
        model.zero_grad()
        outputs = model(X)
        loss = loss_function(outputs, y)

        loss.backward()
        optimizer.step()
        current_loss = loss.item()
        total_loss += current_loss
        progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))
        
    torch.cuda.empty_cache()
    
    val_losses = 0
    precision, recall, f1, accuracy = [], [], [], []
    
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            X, y = data[0].to(device), data[1].to(device)
            outputs = model(X)
            val_losses += loss_function(outputs, y)

            predicted_classes = torch.max(outputs, 1)[1]
            
            for acc, metric in zip((precision, recall, f1, accuracy), 
                                   (precision_score, recall_score, f1_score, accuracy_score)):
                acc.append(
                    calculate_metric(metric, y.cpu(), predicted_classes.cpu())
                )
    
    current_model_accuracy = sum(accuracy)/val_batches
    accuracies.append(current_model_accuracy)
    if current_model_accuracy > best_accuracy:
        best_model = deepcopy(model)
        best_accuracy=current_model_accuracy
        
    print(f"Epoch {epoch+1}/{epochs}, training loss: {total_loss/batches}, validation loss: {val_losses/val_batches}")
    print_scores(precision, recall, f1, accuracy, val_batches)
    losses.append(total_loss/batches)
save_last_model(model)
model_save_name = 'resnet_sgd_best.pkl'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(best_model, path)

print(losses)
print(f"Training time: {time.time()-start_ts}s")


Epoch 1/90, training loss: 1.5392260045639232, validation loss: 1.4858535528182983
	     precision: 0.9786
	        recall: 0.9775
	            F1: 0.9778
	      accuracy: 0.9781



Epoch 2/90, training loss: 1.4791650932226608, validation loss: 1.4763234853744507
	     precision: 0.9864
	        recall: 0.9861
	            F1: 0.9861
	      accuracy: 0.9862



Epoch 3/90, training loss: 1.4729896313600195, validation loss: 1.4754246473312378
	     precision: 0.9865
	        recall: 0.9865
	            F1: 0.9864
	      accuracy: 0.9865



Epoch 4/90, training loss: 1.4709592025671432, validation loss: 1.47457754611969
	     precision: 0.9869
	        recall: 0.9869
	            F1: 0.9868
	      accuracy: 0.9869



Epoch 5/90, training loss: 1.4686612203431282, validation loss: 1.4750540256500244
	     precision: 0.9871
	        recall: 0.9868
	            F1: 0.9868
	      accuracy: 0.9869


...


Epoch 86/90, training loss: 1.4619477588230614, validation loss: 1.468808650970459
	     precision: 0.9924
	        recall: 0.9922
	            F1: 0.9923
	      accuracy: 0.9924



Epoch 87/90, training loss: 1.461950037270975, validation loss: 1.468656301498413
	     precision: 0.9929
	        recall: 0.9928
	            F1: 0.9928
	      accuracy: 0.9929



Epoch 88/90, training loss: 1.4619400572420946, validation loss: 1.4685810804367065
	     precision: 0.9925
	        recall: 0.9924
	            F1: 0.9924
	      accuracy: 0.9925



Epoch 89/90, training loss: 1.461947482786199, validation loss: 1.4685943126678467
	     precision: 0.9930
	        recall: 0.9929
	            F1: 0.9929
	      accuracy: 0.9930



Epoch 90/90, training loss: 1.4619468404794298, validation loss: 1.468474268913269
	     precision: 0.9932
	        recall: 0.9930
	            F1: 0.9931
	      accuracy: 0.9932
[1.5392260045639232, 1.4791650932226608, 1.4729896313600195, 1.4709592025671432, 1.4686612203431282, 1.4674271462060242, 1.4666151914006866, 1.4659180026064549, 1.4653810857455614, 1.4646790678313037, 1.4649907843644685, 1.4641684446253502, 1.4636683212414479, 1.4632786545417964, 1.4631749101793334, 1.463330558368138, 1.462880331824329, 1.4628208179209532, 1.4628090777122644, 1.4626344261901465, 1.462607650360319, 1.4625243362841576, 1.4625405258715534, 1.4625472956374763, 1.4624905293938448, 1.4623249881048954, 1.4625074672800646, 1.4622607653074935, 1.46216947183426, 1.4621133852615031, 1.4620711572134673, 1.4620731756377068, 1.462037784712655, 1.4620120962545562, 1.4620212237718009, 1.462016803369339, 1.4619982623850614, 1.4619991568359993, 1.4619889297465016, 1.4619848987441073, 1.4619933018552216, 1.4619

In [13]:
best_accuracy

0.9931999999999999

In [14]:
accuracies

[0.9781000000000001,
 0.9862,
 0.9865000000000002,
 0.9869,
 0.9869,
 0.9872,
 0.9885999999999999,
 0.9886999999999999,
 0.9907,
 0.9883,
 0.9888000000000001,
 0.991,
 0.9891,
 0.9902000000000001,
 0.9901,
 0.9917,
 0.9911999999999999,
 0.9915,
 0.9923000000000002,
 0.9930000000000001,
 0.9926,
 0.9924,
 0.9924,
 0.9905000000000002,
 0.9921000000000001,
 0.9928000000000001,
 0.9914,
 0.9923000000000002,
 0.992,
 0.9917999999999999,
 0.9921,
 0.9918999999999999,
 0.9924,
 0.9923,
 0.9924,
 0.9926,
 0.992,
 0.9927000000000001,
 0.9928000000000001,
 0.9921000000000001,
 0.9927000000000001,
 0.9926,
 0.9931999999999999,
 0.9929,
 0.9928999999999999,
 0.9929,
 0.9926,
 0.9928000000000001,
 0.9931000000000001,
 0.9931000000000001,
 0.9926999999999999,
 0.993,
 0.9926999999999999,
 0.9924,
 0.9928999999999999,
 0.9928999999999999,
 0.9927999999999999,
 0.9926999999999999,
 0.9927000000000001,
 0.9925999999999998,
 0.9928000000000001,
 0.9926999999999999,
 0.9926,
 0.9926,
 0.9928000000000001,

In [12]:
model

MnistResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=Tru

In [ ]:
print(model)